In [1]:
import cv2
import numpy as np
import os
import random
from sklearn.model_selection import train_test_split
from shutil import copyfile

In [2]:
# === CONFIG ===
input_dir = r"E:\Code\project\Chana\Code\Image Denoising via Outlier Detection using CNN-U-Net Hybrid Model\images_test"
output_dir = r"E:\Code\project\Chana\Code\Image Denoising via Outlier Detection using CNN-U-Net Hybrid Model\images_test\noisy"
clean_output_dir = r"E:\Code\project\Chana\Code\Image Denoising via Outlier Detection using CNN-U-Net Hybrid Modelimages_test\clean"
split_ratios = (0.7, 0.15, 0.15)  # Train, Val, Test
img_size = (64, 64)

In [3]:
# === CREATE OUTPUT FOLDERS ===
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(output_dir, split), exist_ok=True)
    os.makedirs(os.path.join(clean_output_dir, split), exist_ok=True)

In [4]:
# === NOISE FUNCTIONS ===
def add_gaussian_noise(image, mean=0, std=25):
    noise = np.random.normal(mean, std, image.shape).astype(np.float32)
    noisy = image.astype(np.float32) + noise
    return np.clip(noisy, 0, 255).astype(np.uint8)

def add_salt_pepper_noise(image, amount=0.02, salt_vs_pepper=0.5):
    noisy = image.copy()
    total_pixels = image.shape[0] * image.shape[1]
    num_salt = int(total_pixels * amount * salt_vs_pepper)
    num_pepper = int(total_pixels * amount * (1 - salt_vs_pepper))

    for _ in range(num_salt):
        i, j = random.randint(0, noisy.shape[0] - 1), random.randint(0, noisy.shape[1] - 1)
        noisy[i, j] = [255, 255, 255]

    for _ in range(num_pepper):
        i, j = random.randint(0, noisy.shape[0] - 1), random.randint(0, noisy.shape[1] - 1)
        noisy[i, j] = [0, 0, 0]

    return noisy

In [5]:
def add_motion_blur(image, kernel_size=9):
    kernel = np.zeros((kernel_size, kernel_size))
    kernel[kernel_size // 2, :] = np.ones(kernel_size)
    kernel /= kernel_size
    return cv2.filter2D(image, -1, kernel)

In [6]:
# === LOAD & SPLIT ===
image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
if len(image_files) == 0:
    raise ValueError(f"No valid images found in {input_dir}")
    
train_files, val_test = train_test_split(image_files, train_size=split_ratios[0], random_state=42)
val_files, test_files = train_test_split(val_test, test_size=split_ratios[2]/(split_ratios[1]+split_ratios[2]), random_state=42)

In [7]:
splits = {
    'train': train_files,
    'val': val_files,
    'test': test_files
}

In [8]:
# === PROCESS IMAGES ===
for split_name, files in splits.items():
    print(f"\nProcessing {split_name.upper()} ({len(files)} images):")
    for filename in files:
        img_path = os.path.join(input_dir, filename)
        image = cv2.imread(img_path)
        
        if image is None:
            print(f"[✖] Error reading {img_path}")
            continue

        image = cv2.resize(image, img_size)

        # Save the clean image
        clean_save_filename = f"{split_name}_{os.path.splitext(filename)[0]}.jpg"
        clean_save_path = os.path.join(clean_output_dir, split_name, clean_save_filename)
        
        success_clean = cv2.imwrite(clean_save_path, image)
        if success_clean:
            print(f"[✔] Saved clean image: {clean_save_path}")
        else:
            print(f"[✖] Failed to save clean image: {clean_save_path}")

        # Random noise type
        noise_type = random.choice(['gaussian', 'salt_pepper', 'motion_blur'])
        if noise_type == 'gaussian':
            noisy = add_gaussian_noise(image)
        elif noise_type == 'salt_pepper':
            noisy = add_salt_pepper_noise(image)
        else:
            noisy = add_motion_blur(image)

        # Save noisy image
        noisy_save_filename = f"{split_name}_{os.path.splitext(filename)[0]}.jpg"
        noisy_save_path = os.path.join(output_dir, split_name, noisy_save_filename)

        success_noisy = cv2.imwrite(noisy_save_path, noisy)
        if success_noisy:
            print(f"[✔] Saved noisy image: {noisy_save_path} ({noise_type})")
        else:
            print(f"[✖] Failed to save noisy image: {noisy_save_path}")



Processing TRAIN (1400 images):
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_cat_851.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_cat_851.jpg (salt_pepper)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_cat_616.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_cat_616.jpg (salt_pepper)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_cat_60.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_cat_60.jpg (salt_pepper)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_dog_31.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_dog_31.jpg (motion_blur)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_dog_422.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_dog_422

[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_cat_733.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_cat_733.jpg (salt_pepper)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_dog_752.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_dog_752.jpg (motion_blur)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_dog_73.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_dog_73.jpg (gaussian)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_dog_664.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_dog_664.jpg (motion_blur)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_dog_638.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_dog_638.jpg (motion_blur)
[✔] Saved clean

[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_cat_700.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_cat_700.jpg (motion_blur)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_cat_126.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_cat_126.jpg (gaussian)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_cat_945.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_cat_945.jpg (gaussian)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_cat_749.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_cat_749.jpg (gaussian)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_cat_994.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_cat_994.jpg (salt_pepper)
[✔] Saved clean ima

[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_cat_317.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_cat_317.jpg (motion_blur)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_dog_400.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_dog_400.jpg (motion_blur)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_dog_159.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_dog_159.jpg (gaussian)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_dog_526.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_dog_526.jpg (salt_pepper)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_cat_38.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_cat_38.jpg (salt_pepper)
[✔] Saved clean

[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_cat_224.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_cat_224.jpg (gaussian)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_dog_976.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_dog_976.jpg (motion_blur)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_dog_403.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_dog_403.jpg (gaussian)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_dog_816.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_dog_816.jpg (salt_pepper)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\train\train_dog_615.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\train\train_dog_615.jpg (motion_blur)
[✔] Saved clean 

...

[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_cat_313.jpg (salt_pepper)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_dog_179.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_dog_179.jpg (salt_pepper)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_dog_607.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_dog_607.jpg (motion_blur)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_cat_201.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_cat_201.jpg (salt_pepper)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_cat_826.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_cat_826.jpg (motion_blur)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_cat_993.jpg
[✔] Saved noisy image: E:\Code

[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_cat_621.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_cat_621.jpg (motion_blur)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_dog_337.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_dog_337.jpg (salt_pepper)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_dog_936.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_dog_936.jpg (motion_blur)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_dog_688.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_dog_688.jpg (gaussian)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_dog_361.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_dog_361.jpg (gaussian)
[✔] Saved clean image: E:\Code\proje

[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_cat_649.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_cat_649.jpg (gaussian)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_cat_544.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_cat_544.jpg (motion_blur)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_cat_947.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_cat_947.jpg (gaussian)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_dog_522.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_dog_522.jpg (gaussian)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_dog_217.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_dog_217.jpg (gaussian)
[✔] Saved clean image: E:\Code\project\Cha

[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_dog_646.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_dog_646.jpg (gaussian)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_cat_588.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_cat_588.jpg (salt_pepper)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_cat_188.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_cat_188.jpg (motion_blur)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_dog_490.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_dog_490.jpg (motion_blur)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_cat_936.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_cat_936.jpg (salt_pepper)
[✔] Saved clean image: E:\Code\pr

[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_dog_652.jpg (salt_pepper)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_dog_344.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_dog_344.jpg (gaussian)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_cat_840.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_cat_840.jpg (gaussian)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_cat_333.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_cat_333.jpg (motion_blur)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_cat_554.jpg
[✔] Saved noisy image: E:\Code\project\Chana\Code\images_test\noisy\test\test_cat_554.jpg (motion_blur)
[✔] Saved clean image: E:\Code\project\Chana\Code\images_test\clean\test\test_cat_318.jpg
[✔] Saved noisy image: E:\Code\proje

In [9]:
print("✅ All images processed, split, and saved.")

✅ All images processed, split, and saved.
